In [1]:
%matplotlib nbagg
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
import os
import scipy.io as sio
path = os.getcwd()

This function should assign the points in your dataMatrix to the closest centroid, the distance is computed using the L2 norm.

In [2]:
def assignPoints(dataMatrix, centroids):
    centroidIndices = np.zeros(dataMatrix.shape[1],dtype=np.int)
    for i in range(dataMatrix.shape[1]):
        for j in range(centroids.shape[1]):
            d = np.linalg.norm((dataMatrix[:,i]-centroids[:,j]), 2)
            if j==0:
                centroidIndex = 0
                centroidDistance = d
            elif d < centroidDistance:
                centroidIndex = j
                centroidDistance = d
        centroidIndices[i] = centroidIndex
    return centroidIndices

This function should update the centroids to its assigned points.

In [3]:
def updateCentroids(dataMatrix, assignedPoints):
    nrCentroids = assignedPoints.max()+1    
    updatedCentroids = np.zeros((dataMatrix.shape[0],nrCentroids))
    for i in range(nrCentroids):
        updatedCentroids[:,i] = dataMatrix.T[assignedPoints==i].mean(0)
    return updatedCentroids

This function should compute the cost function.

In [4]:
def computeCost(dataMatrix, centroids, assignedPoints):
    cost = 0
    for i in range(centroids.shape[1]):
        clusterMatrix = dataMatrix.T[assignedPoints==i] - centroids[:,i]
        for dataPoint in clusterMatrix:
            cost = cost + np.linalg.norm(dataPoint, 2)
    return cost

Run Kmeans Algorithm:

In [5]:
dataMatrix = sio.loadmat(os.path.join(path,'data','toydata2.mat'))['data']
numberOfCluster = 3
Tol=1e-6

In [6]:
centroids = dataMatrix[:,np.random.choice(dataMatrix.shape[1],numberOfCluster,replace=False)] # initialize Centroids with random samples
cost = []
cluster=[]
while True: # convergence criterion
    assignedPoints = assignPoints(dataMatrix, centroids)
    iter_cluster=[]
    for j in range(numberOfCluster):
        iter_cluster.append(dataMatrix.T[assignedPoints==j].T)
    cluster.append(iter_cluster)
    centroids = updateCentroids(dataMatrix, assignedPoints)
    cost.append(computeCost(dataMatrix, centroids, assignedPoints))
    if len(cost)>1:
        RE = (cost[-1]-cost[-2])/max(cost[-1],cost[-2])
        if abs(RE) < Tol:
            break

In [7]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
def animate(i,data):
    ax.clear()
    colors = iter(cm.rainbow(np.linspace(0, 1, len(data[0]))))
    for A in data[i%len(data)]:
        ax.scatter(A[0],A[1],A[2],c=next(colors),marker='o')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Iteration: '+str(i%len(data)+1))
ani = animation.FuncAnimation(fig, animate, fargs=[cluster], interval=1000)
plt.show()

<IPython.core.display.Javascript object>